# SVD 奇异值分解
奇异值分解(Singular Value Decomposition：
> 优点：简化数据，去除噪声，提高算法运行效果<br>
> 缺点：数据的转换可能难以理解<br>
> 适用数据类型：数值型数据

假设矩阵A是m*n的，奇异值分解的结果就是将矩阵A分解为如下三个矩阵的乘积：
$$A_{mxn} = U_{mxm}\Sigma_{mxn}{V^T_{nxn}}$$
U 的列由 $AA^T$ 的单位化过的特征向量构成<br>
V 的列由 $A^TA$ 的单位化过的特征向量构成<br>
Σ 的对角元素来源于 $AA^T$ 或 $A^TA$ 的特征值的平方根，并且是按从大到小的顺序排列的。奇异值就是 Σ 对角线上的元素。

## 1. 使用Python实现SVD

In [6]:
import numpy as np

In [7]:
U, sigma, VT = np.linalg.svd([[1, 1], [7, 7]])
print(U, '\n\n', sigma, '\n\n', VT)

[[-0.14142136 -0.98994949]
 [-0.98994949  0.14142136]] 

 [1.00000000e+01 2.82797782e-16] 

 [[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]]


In [32]:
def loadExData():
    return [[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]]

In [33]:
Data = loadExData()
U, Sigma, VT = np.linalg.svd(Data)
Sigma

array([9.72140007e+00, 5.29397912e+00, 6.84226362e-01, 4.11502614e-16,
       1.36030206e-16])

可以看到前三个数值比后面的大了很多，近似可以把后面两个数据看成0.

In [34]:
# 重构原矩阵
Sig3 = np.mat([[Sigma[0], 0, 0], [0, Sigma[1], 0], [0, 0, Sigma[2]]])
U[:, :3] * Sig3 * VT[:3, :]

matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          7.75989921e-16,  7.71587483e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          3.00514919e-16,  2.77832253e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          2.18975112e-16,  2.07633779e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          3.00675663e-17, -1.28697294e-17],
        [ 1.00000000e+00,  1.00000000e+00, -5.48397422e-16,
          2.00000000e+00,  2.00000000e+00],
        [ 3.21319929e-16,  4.43562065e-16, -3.48967188e-16,
          3.00000000e+00,  3.00000000e+00],
        [ 9.71445147e-17,  1.45716772e-16, -1.52655666e-16,
          1.00000000e+00,  1.00000000e+00]])

## 2. 基于协同过滤的推荐引擎
### a. 相似度计算
欧氏距离、皮尔逊相似度、余弦相似度

In [35]:
# 计算欧氏距离
def  ecludSim(inA, inB):
    return 1.0 / (1.0 + np.linalg.norm(inA - inB))

# 计算皮尔逊距离
def pearsSim(inA, inB):
    # 如果不存在，则返回1.0，这时完全相关
    if len(inA) < 3:
        return 1.0
    return 0.5 + 0.5 * np.corrcoef(inA, inB, rowvar=0)[0][1]

# 计算余弦相似度，夹角为90相似度为0.
def cosSim(inA, inB):
    num = float(inA.T * inB)
    denom = np.linalg.norm(inA) * np.linalg.norm(inB)
    return 0.5 + 0.5 * (num / denom)

In [36]:
myMat = np.mat(loadExData())
ecludSim(myMat[:, 0], myMat[:, 4])

0.13367660240019172

In [37]:
ecludSim(myMat[:, 0], myMat[:, 0])

1.0

In [38]:
pearsSim(myMat[:, 0], myMat[:, 4]), pearsSim(myMat[:, 0], myMat[:, 1])

(0.23768619407595815, 1.0)

### b. 餐馆菜肴推荐引擎
推荐系统的工作过程是:给定一个用户,系统会为此用户返回N个最好的推荐菜。为了实现这一点,则需要我们做到:
    1. 寻找用户没有评级的菜肴,即在用户-物品矩阵中的 0 值;
    2. 在用户没有评级的所有物品中,对每个物品预计一个可能的评级分数。这就是说,我们认为用户可能会对物品的打分(这就是相似度计算的初衷);
    3. 对这些物品的评分从高到低进行排序,返回前N个物品。

In [39]:
def standEst(dataMat, user, simMeas, item):
    """
    计算用户未评分物品中，以对该物品和其他物品评分的相似度，然后对未评分的物品评分
    
    Parameters
    -----------
        dataMat :      训练数据集
        user :         用户编号
        simMeas :      相似度计算方法
        item :         未评分的物品编号
    Returns
    -----------
        ratSimTotal/simTotal : 评分（0~5之间）
    """
    n = dataMat.shape[1]
    # 初始化两个评分值
    simTotal = 0.0
    ratSimTotal = 0.0
    # 遍历用户对所有物品的评分
    for j in range(n):
        userRating = dataMat[user, j]
        # 用户对该物品评分值为0，跳过该物品，继续循环
        if userRating == 0:
            continue
        # 寻找两个用户都评级的物品
        # overlap计算出对当前物品和其他物品都给出评分的用户ID
        overLap = np.nonzero(np.logical_and(dataMat[:, item].A > 0,
                                            dataMat[:, j].A > 0))[0]
        # 如果所有用户对item和第j个物品的评分都没有重合，那么定义相似度为0
        if len(overLap) == 0:
            similarity = 0
        # 否则基于用户对item和第j个物品的评分的重合部分计算相似度
        else:
            similarity = simMeas(dataMat[overLap, item], dataMat[overLap, j])
        print("The %d and %d similarity is: %f" % (item, j, similarity))
        # 累加相似度
        simTotal += similarity
        # 计算相似度和用户评分乘积
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        # 归一化，将评分值限制在0~5之间
        return ratSimTotal / simTotal
    
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    """
    返回评分最高的N个物品
    """
    # 建立一个未评分物品列表
    unratedItems = np.nonzero(dataMat[user, :].A == 0)[1]
    if len(unratedItems) == 0:
        return 'you rated everything'
    # 物品的编号和评分值
    itemScores = []
    for item in unratedItems:
        # 获取当前物品评分
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
        # 排序后返回并取前N个结果
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [97]:
myMat = np.mat([[4, 4, 0, 2, 2],
         [4, 0, 0, 3, 3],
         [4, 0, 0, 1, 1],
         [1, 1, 1, 2, 0],
         [2, 2, 2, 0, 0],
         [1, 1, 1, 0, 0],
         [5, 5, 5, 0, 0]])

myMat[0, 1] = myMat[0, 0] = myMat[1, 0] = myMat[2, 0] = 4
myMat[3, 3] = 2
myMat

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [1, 1, 1, 0, 0],
        [5, 5, 5, 0, 0]])

In [98]:
standEst(myMat, 2, ecludSim, 1)

The 1 and 0 similarity is: 1.000000
The 1 and 3 similarity is: 0.309017
The 1 and 4 similarity is: 0.333333


2.8266504712098603

In [107]:
recommend(myMat, 2)

The 1 and 0 similarity is: 1.000000
The 1 and 3 similarity is: 0.928746
The 1 and 4 similarity is: 1.000000
The 2 and 0 similarity is: 1.000000
The 2 and 3 similarity is: 1.000000
The 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0243290220056256)]

In [108]:
recommend(myMat, 2, simMeas=ecludSim)

The 1 and 0 similarity is: 1.000000
The 1 and 3 similarity is: 0.309017
The 1 and 4 similarity is: 0.333333
The 2 and 0 similarity is: 1.000000
The 2 and 3 similarity is: 0.500000
The 2 and 4 similarity is: 0.000000


[(2, 3.0), (1, 2.8266504712098603)]

In [109]:
recommend(myMat, 2, simMeas=pearsSim)

The 1 and 0 similarity is: 1.000000
The 1 and 3 similarity is: 1.000000
The 1 and 4 similarity is: 1.000000
The 2 and 0 similarity is: 1.000000
The 2 and 3 similarity is: 1.000000
The 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0)]

### c. 利用SVD提高推荐效果

In [110]:
def loadExData2():
    # 书上代码给的示例矩阵
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

In [111]:
U, Sigma, VT = np.linalg.svd(np.mat(loadExData2()))
Sigma

array([15.77075346, 11.40670395, 11.03044558,  4.84639758,  3.09292055,
        2.58097379,  1.00413543,  0.72817072,  0.43800353,  0.22082113,
        0.07367823])

In [112]:
Sig2 = Sigma ** 2
sum(Sig2)

541.9999999999994

In [113]:
sum(Sig2) * 0.9

487.7999999999995

In [114]:
sum(Sig2[:2])

378.8295595113579

In [115]:
sum(Sig2[:3])

500.5002891275793

该值高于总能量的 90% ,这就可以了。于是,我们可以将一个 11 维的矩阵转换成一个 3 维的矩阵。

In [116]:
def svdEst(dataMat, user, simMeas, item):
    """
    svdEst
    
    Parameters
    -----------
        dataMat :      训练数据集
        user :         用户编号
        simMeas :      相似度计算方法
        item :         未评分的物品编号
    Returns
    -----------
        ratSimTotal/simTotal : 评分（0~5之间）
    """
    n = dataMat.shape[1]
    simTotal = 0.0
    ratSimTotal = 0.0
    # 奇异值分解
    # 在SVD分解之后，我们只利用了包含90%能量的奇异值
    U, Sigma, VT = np.linalg.svd(dataMat)
    # analyse_data(Sigma, 20)
    # 使用这些奇异值构造出一个对角阵
    Sig4 = np.mat(np.eye(4) * Sigma[:4])
    # 利用U矩阵将物品转换到低维空间中，构建转换后的物品（物品：4个主要特征，包含了90%的能量）
    xformedItems = dataMat.T * U[:, :4] * Sig4.I
    print('dataMat', dataMat.shape)
    print('U[:, :4]', U[:, :4].shape)
    print('Sig4.I', Sig4.I.shape)
    print('VT[:4, :]', VT[:4, :].shape)
    print('xformedItems', xformedItems.shape)
    
    for j in range(n):
        userRating = dataMat[user, j]
        if userRating == 0:
            continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        print("the %d and %d similarity is: %f" % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal/simTotal

In [117]:
recommend(myMat, 1, estMethod=svdEst)

dataMat (7, 5)
U[:, :4] (7, 4)
Sig4.I (4, 4)
VT[:4, :] (4, 5)
xformedItems (5, 4)
the 1 and 0 similarity is: 0.498142
the 1 and 3 similarity is: 0.498131
the 1 and 4 similarity is: 0.509974
dataMat (7, 5)
U[:, :4] (7, 4)
Sig4.I (4, 4)
VT[:4, :] (4, 5)
xformedItems (5, 4)
the 2 and 0 similarity is: 0.552670
the 2 and 3 similarity is: 0.552976
the 2 and 4 similarity is: 0.217301


[(2, 3.417756918659238), (1, 3.330717154558564)]

## 3. 基于SVD的图像压缩

In [128]:
# 图像压缩函数
# 加载并转换数据
def imgLoadData(filename):
    myl = []
    # 打开文本文件，并从文件以数组方式读入字符
    for line in open(filename).readlines():
        newRow = []
        for i in range(32):
            newRow.append(int(line[i]))
        myl.append(newRow)
    # 矩阵调入后，就可以在屏幕上输出该矩阵
    myMat = np.mat(myl)
    return myMat

def printMat(inMat, thresh=0.8):
    # 为了显示深色浅色，当大于阈值时打印1，小于阈值时打印0
    for i in range(32):
        for k in range(32):
            if float(inMat[i, k]) > thresh:
                print(1)
            else:
                print(0)
        print('')

# 实现图像压缩
def imgCompress(numSV=3, thresh=0.8):
    # 构建一个列表
    myMat = imgLoadData('0_5.txt')
    print("***original matrix******")
    # 重构图像
    printMat(myMat, thresh)
    # SVD分解
    U, Sigma, VT = np.linalg.svd(myMat)
    
    # 新建一个0矩阵，对角线元素填充为奇异值
    SigRecon = np.mat(np.zeros((numSV, numSV)))
    for k in range(numSV):
        SigRecon[k, k] = Sigma[k]
    # 输出压缩后的矩阵
    reconMat = U[:, :numSV] * SigRecon * VT[:numSV, :]
    print("****reconstructed matrix using %d singular values******" % numSV)
    printMat(reconMat, thresh)

In [129]:
imgCompress(2)

***original matrix******
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0

0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0

0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0

0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0

0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0

0
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0

0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0

